In [4]:
import pandas as pd
import numpy as np


def run_multiple_experiments(data, target_column, feature_column, save_path, num_runs=50, number_of_quantile_one_end=4):
    results = []
    for run in range(num_runs):
        print(f"Starting run {run + 1}/{num_runs}")
        # Assuming QNNs_gen is a function you have defined elsewhere that computes some results
        result = QNNs_gen(data, target_column, feature_column, save_path, number_of_quantile_one_end)
        results.append(result)
    
    # Calculating quantiles
    # For the bottom quantile from 0 to 0.5 divided by 'number_of_quantile_one_end'
    bottom_quantile = [i / (number_of_quantile_one_end * 2) for i in range(number_of_quantile_one_end)]
    # For the top quantile from 0.5 to 1 divided by 'number_of_quantile_one_end'
    top_quantile = [0.5 + i / (number_of_quantile_one_end * 2) for i in range(number_of_quantile_one_end)]
    
    # Sort bottom quantile in descending order
    bottom_quantile = sorted(bottom_quantile, reverse=True)
    # Sort top quantile in ascending order (it's already ascending but included for clarity)
    top_quantile = sorted(top_quantile)

    # Combine quantiles starting from 0.5 and interleaving top and bottom quantiles
    quantiles = [0.5]
    for t, b in zip(top_quantile, bottom_quantile):
        quantiles.append(t)
        quantiles.append(b)
    
    # Create a DataFrame from the results with appropriate quantile columns
    print(f'in csv {quantiles}')
    df = pd.DataFrame(results, columns=quantiles)
    df.to_csv(f"{save_path}quantile_performance_numberofquantile{number_of_quantile_one_end*2 + 1}.csv", index=False)
    print("All runs completed and results saved to CSV.")

def QNNs_gen(data,target_column,feature_column,save_path,number_of_quantile_one_end):
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import DataLoader, TensorDataset
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score
    import joblib
    class NeuralNet(nn.Module):
        def __init__(self, input_size, output_size):
            super(NeuralNet, self).__init__()
            self.layer1 = nn.Linear(input_size, 128)
            self.relu1 = nn.ReLU()
            self.layer2 = nn.Linear(128, 64)
            self.relu2 = nn.ReLU()
            self.layer3 = nn.Linear(64, 32)
            self.relu3 = nn.ReLU()
            self.output_layer = nn.Linear(32, output_size)

        def forward(self, x):
            # Forward pass records for optimization
            self.outputs = {}
            self.activations = {}

            z1 = self.layer1(x)
            self.activations['layer1'] = z1
            x = self.relu1(z1)

            z2 = self.layer2(x)
            self.activations['layer2'] = z2
            x = self.relu2(z2)

            z3 = self.layer3(x)
            self.activations['layer3'] = z3
            x = self.relu3(z3)

            out = self.output_layer(x)
            self.outputs['output'] = out
            return out
    def adjusted_quantile_loss(outputs, targets, quantile, lower_bound=None, upper_bound=None):
        errors = targets - outputs
        basic_loss = torch.max((quantile - 1) * errors, quantile * errors)
        loss = torch.mean(basic_loss)

        if lower_bound is not None:
            crossing_penalty = torch.mean(torch.relu(lower_bound - outputs))
            loss += crossing_penalty
        if upper_bound is not None:
            crossing_penalty = torch.mean(torch.relu(outputs - upper_bound))
            loss += crossing_penalty

        return loss
    features = data[feature_column]
    target = data[target_column]
    X = torch.tensor(features.values, dtype=torch.float32)
    y = torch.tensor(target.values, dtype=torch.float32).unsqueeze(1)
    scaler = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train = torch.tensor(scaler.fit_transform(X_train), dtype=torch.float32)
    X_test = torch.tensor(scaler.transform(X_test), dtype=torch.float32)

    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # For the bottom quantile from 0 to 0.5 divided by 'number_of_quantile_one_end'
    bottom_quantiles = [i / (number_of_quantile_one_end * 2) for i in range(number_of_quantile_one_end)]
    # For the top quantile from 0.5 to 1 divided by 'number_of_quantile_one_end'
    top_quantiles = [0.5 + i / (number_of_quantile_one_end * 2) for i in range(number_of_quantile_one_end)]
    # Sort bottom quantile in descending order
    bottom_quantiles = sorted(bottom_quantiles, reverse=True)
    # Sort top quantile in ascending order (it's already ascending but included for clarity)
    top_quantiles = sorted(top_quantiles)

    # Combine quantiles starting from 0.5 and interleaving top and bottom quantiles
    quantiles = [0.5]
    for t, b in zip(top_quantiles, bottom_quantiles):
        quantiles.append(t)
        quantiles.append(b)
    models = [NeuralNet(X_train.shape[1], 1) for _ in quantiles]
    optimizers = [optim.Adam(model.parameters(), lr=0.01) for model in models]

    # Store bounds for each quantile and input
    input_bounds = {q: {'lower': None, 'upper': None} for q in quantiles}



    # Adjusted training loop
    print(print(f'in function {quantiles}'))
    for epoch in range(200):
        for inputs, labels in train_loader:
            current_outputs = {}  # Dictionary to store outputs for current batch

            # Clear gradients at the start of each batch
            for optimizer in optimizers:
                optimizer.zero_grad()

            first_pass = True  # Flag to handle retain_graph properly

            for q_idx, quantile in enumerate(quantiles):
                model = models[q_idx]
                outputs = model(inputs)
                current_outputs[quantile] = outputs.detach().clone()  # Detach and clone to avoid in-place modifications
                def set_bounds(top_quantiles, bottom_quantiles, current_outputs, quantile):
                    lower_bound = None
                    upper_bound = None

                    # Check if the quantile is in the top or bottom array
                    if quantile in top_quantiles:
                        # Find the index of the quantile in the top array
                        index = top_quantiles.index(quantile)
                        # Set lower bound from the previous quantile or 0.5 if it is the first quantile in the array
                        lower_bound = current_outputs.get(top_quantiles[index - 1] if index > 0 else 0.5)

                    elif quantile in bottom_quantiles:
                        # Find the index of the quantile in the bottom array
                        index = bottom_quantiles.index(quantile)
                        # Set upper bound from the previous quantile or 0.5 if it is the first quantile in the array
                        upper_bound = current_outputs.get(bottom_quantiles[index - 1] if index > 0 else 0.5)

                    return lower_bound, upper_bound

                lower_bound, upper_bound=set_bounds(top_quantiles, bottom_quantiles, current_outputs, quantile)

                # Adjusted quantile loss calculation
                loss = adjusted_quantile_loss(outputs, labels, quantile, lower_bound, upper_bound)
                loss.backward(retain_graph=first_pass)  # Only retain graph on the first backward pass

                first_pass = False  # Ensure subsequent passes do not retain graph
            for optimizer in optimizers:
                optimizer.step()

            # Optionally, clear dictionary to free memory after processing each batch
            current_outputs.clear()
    # Evaluation of models
    all_rquare=[]
    for i, model in enumerate(models):
        model.eval()
        with torch.no_grad():
            predictions_train = model(X_train)
            predictions_test = model(X_test)
            r2_train = r2_score(y_train.numpy(), predictions_train.numpy())
            r2_test = r2_score(y_test.numpy(), predictions_test.numpy())
            all_rquare.append(r2_test)
            print(f'Quantile: {quantiles[i]}, Training R-squared: {r2_train:.4f}, Testing R-squared: {r2_test:.4f}')
    # After training all models, save each model's state
    joblib.dump(scaler, f'{save_path}scaler.pkl')
    for idx, model in enumerate(models):
        torch.save({
            'state_dict': model.state_dict(),
            'weights': [param.detach().numpy() for param in model.parameters()]
        }, f'{save_path}model_quantile_{quantiles[idx]}.pth')
    print(f'R sqaures {all_rquare}')
    

    return all_rquare
file_path = '/Users/ansonkong/Downloads/QNN-test/p122_synthetic.csv'
data = pd.read_csv(file_path)
#Feature
feature_column=['battery_2#p122','gas-cc#p122','upv#p122','wind-ons#p122']
#target
target_column='value'
save_path="/Users/ansonkong/Downloads/QNN-test/"
# Example usage
data = pd.read_csv('/Users/ansonkong/Downloads/QNN-test/p122_synthetic.csv')  # Load your data
run_multiple_experiments(data, target_column, feature_column, '/Users/ansonkong/Downloads/QNN-test/')


Starting run 1/50
in function [0.5, 0.5, 0.375, 0.625, 0.25, 0.75, 0.125, 0.875, 0.0]
None


KeyboardInterrupt: 